In [1]:
import string

import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np
import os

# Utils

In [2]:
def find_files(path, ext):
    files = []
    for file in os.listdir(path):
        if file.endswith(ext):
            files.append(file)
    return files

In [3]:
from typing import Iterable, Union

def get_most_impacting(report: dict, n=2):
    """
    Get the most impacting tokens from experiments.

    Args:
        report (dict): The input dictionary containing experiment data >> dict_keys(['idx', 'tokens', 'importance', 'logits', 'predicted_label', 'true_label'])
        n (int, optional): The number of tokens to consider. Defaults to 2.

    Returns:
        dict: A dictionary containing analysis results.
    """
    _report = {
        "predict": dict(),
        "true": dict(),
        "at_all": dict(),
    }
    for pred, true, importance, tokens in zip(
        report["predicted_label"],
        report["true_label"],
        report["importance"],
        report["tokens"]
    ):

        _iter_pred = zip(np.array(tokens), importance[pred])
        data_pred = sorted([(token, importance) for token, importance in _iter_pred],
                           key= lambda x: x[1],
                           reverse=True)
        data_pred = data_pred[:n]

        if pred==true:
            data_true = data_pred.copy()
            data_all = data_pred.copy()
        else:
            _iter_true = zip(np.array(tokens), importance[true])
            data_true = sorted([(token, importance) for token, importance in _iter_true],
                               key= lambda x: x[1],
                               reverse=True)
            data_true = data_true[:n]

            data_all = sorted(data_true + data_pred,
                              key= lambda x: x[1],
                              reverse=True)
            data_all = data_all[:n]

        for p, t, a in zip(data_pred, data_true, data_all):
            if p[1] > 0:
                _report["predict"][p[0]] = _report["predict"].get(p[0], 0) + p[1]
            if t[1] > 0:
                _report["true"][t[0]] = _report["true"].get(t[0], 0) + t[1]
            if a[1] > 0:
                _report["at_all"][a[0]] = _report["at_all"].get(a[0], 0) + a[1]
            
    return {
        "predict": sorted(_report["predict"].items(), key=lambda x: x[1], reverse=True)[:n],
        "true": sorted(_report["true"].items(), key=lambda x: x[1], reverse=True)[:n],
        "at_all": sorted(_report["at_all"].items(), key=lambda x: x[1], reverse=True)[:n],
    }


def get_importance(report: dict, d_tokens: Iterable):
    _report = {}
    for dt in d_tokens:
        _report[dt] = {
            "l0": list(),
            "l1": list(),
            "l2": list(),
        }
    for pred, true, importance, tokens in zip(
        report["predicted_label"],
        report["true_label"],
        report["importance"],
        report["tokens"]
    ):
        for token, importance in zip(tokens, importance):
            token = token.replace('##', '').replace('Ġ', '')
            for dt in d_tokens:
                if dt == token:
                    _report[dt]["l1"].append(importance[1])
                    _report[dt]["l0"].append(importance[0])
                    _report[dt]["l2"].append(importance[2])

            # if pred==true:
            #     if pred > 0:
            #         for dt in d_tokens:
            #             if dt == token:
            #                 _report[dt]["tp"].append(importance[1])
            #     else:
            #         for dt in d_tokens:
            #             if dt == token:
            #                 _report[dt]["tn"].append(importance[0])
            # else:
            #     if pred > 0:
            #         for dt in d_tokens:
            #             if dt == token:
            #                 _report[dt]["fp"].append(importance[1])
            #     else:
            #         for dt in d_tokens:
            #             if dt == token:
            #                 _report[dt]["fn"].append(importance[0])

    return _report

In [4]:
def get_importance_all(report: dict, metric_function=np.mean, split=True):
    _report = {}

    for pred, true, importance, tokens in zip(
        report["predicted_label"],
        report["true_label"],
        report["importance"],
        report["tokens"]
    ):
        for token, importance in zip(tokens, importance):
            if token not in _report:
                _report[token] = {
                    "l1": list(),
                    "l2": list(),
                    "l0": list(),
                }

            _report[token]["l1"].append(importance[1])
            _report[token]["l0"].append(importance[0])
            _report[token]["l2"].append(importance[2])

    # Calculate the metric for each token and store them in a list of tuples
    l0_metrics, l1_metrics, l2_metrics = [], [], []
    for token, values in _report.items():
        l0_metrics.append((token, metric_function(values["l0"])))
        l1_metrics.append((token, metric_function(values["l1"])))
        l2_metrics.append((token, metric_function(values["l2"])))

    # Sort the list of tuples by the metric value
    l0_metrics = sorted(l0_metrics, key=lambda x: x[1], reverse=True)
    l1_metrics = sorted(l1_metrics, key=lambda x: x[1], reverse=True)
    l2_metrics = sorted(l2_metrics, key=lambda x: x[1], reverse=True)

    if split:
        l0_metrics = l0_metrics[:NUMBER_OF_IMPORTANT_BAR]
        l1_metrics = l1_metrics[:NUMBER_OF_IMPORTANT_BAR]
        l2_metrics = l2_metrics[:NUMBER_OF_IMPORTANT_BAR]

    return {
        "l0": l0_metrics,
        "l1": l1_metrics,
        "l2": l2_metrics,
    }


In [5]:
def get_most_popular(report: dict, n=2):
    _report = {
        "predict": dict(),
        "true": dict(),
        "at_all": dict(),
    }
    for pred, true, importance, tokens in zip(
        report["predicted_label"],
        report["true_label"],
        report["importance"],
        report["tokens"]
    ):

        _iter_pred = zip(np.array(tokens), importance[pred])
        data_pred = sorted([(token, importance) for token, importance in _iter_pred],
                           key= lambda x: x[1],
                           reverse=True)
        data_pred = data_pred[:n]

        if pred==true:
            data_true = data_pred.copy()
            data_all = data_pred.copy()
        else:
            _iter_true = zip(np.array(tokens), importance[true])
            data_true = sorted([(token, importance) for token, importance in _iter_true],
                               key= lambda x: x[1],
                               reverse=True)
            data_true = data_true[:n]

            data_all = sorted(data_true + data_pred,
                              key= lambda x: x[1],
                              reverse=True)
            data_all = data_all[:n]

        for p, t, a in zip(data_pred, data_true, data_all):
            if p[1] > 0:
                _report["predict"][p[0]] = _report["predict"].get(p[0], 0) + 1
            if t[1] > 0:
                _report["true"][t[0]] = _report["true"].get(t[0], 0) + 1
            if a[1] > 0:
                _report["at_all"][a[0]] = _report["at_all"].get(a[0], 0) + 1

    return {
        "predict": sorted(_report["predict"].items(), key=lambda x: x[1], reverse=True)[:n],
        "true": sorted(_report["true"].items(), key=lambda x: x[1], reverse=True)[:n],
        "at_all": sorted(_report["at_all"].items(), key=lambda x: x[1], reverse=True)[:n],
    }


In [6]:
def read_report(path):
    report = np.load(path, allow_pickle=True).item()
    report["tokens"] = [[token.replace('##', '').replace('Ġ', '') for token in tokens ]for tokens in report["tokens"]]
    return report


# Analysis

In [7]:
BASE_PATH = "."
DATASET_NAME = "mnli"
EXPERIMENTS_PATH = os.path.join(BASE_PATH, DATASET_NAME)

FILES = find_files(EXPERIMENTS_PATH, ".npy")

NUMBER_OF_IMPORTANT_WC = 300
NUMBER_OF_IMPORTANT_BAR = 15

DEMO = False

In [8]:
reports = {
    file.split('.')[0]: read_report(os.path.join(EXPERIMENTS_PATH, file)) for file in FILES
}

## Word cloud

### Most important

In [9]:
save_wc_path = os.path.join(EXPERIMENTS_PATH, "most_important")
if os.path.exists(save_wc_path):
    print("Path already exists")
else:
    os.mkdir(save_wc_path)
    print("Path created")

Path already exists


In [10]:
for exp_name, report in reports.items():
    rep_imp = get_most_impacting(report, n=NUMBER_OF_IMPORTANT_WC)
    _name = f"{exp_name}_imp.png"
    path = os.path.join(save_wc_path, _name)
    fig, axs = plt.subplots(1, len(rep_imp), figsize=(4*len(rep_imp), 4))
    for i, k in enumerate(rep_imp.keys()):
        wordcloud = WordCloud(width=800, height=800, background_color="white")
        wordcloud.generate_from_frequencies(dict(rep_imp[k]))

        axs[i].imshow(wordcloud, interpolation="bilinear")
        axs[i].axis("off")

        axs[i].set_title(k)

    fig.suptitle(exp_name, fontsize=16)
    plt.tight_layout(pad=1.0)
    if DEMO:
        plt.show()
    else:
        plt.savefig(path, format="png")
    plt.close()

In [11]:
for exp_name, report in reports.items():
    rep_imp = get_most_impacting(report, n=NUMBER_OF_IMPORTANT_BAR)
    _name = f"{exp_name}_imp_bar.png"
    path = os.path.join(save_wc_path, _name)
    fig, axs = plt.subplots(1, len(rep_imp), figsize=(6*len(rep_imp), 6))

    for i, k in enumerate(rep_imp.keys()):

        tokens = [item[0] for item in rep_imp[k]]
        freqs = [item[1] for item in rep_imp[k]]

        axs[i].bar(tokens, freqs)
        axs[i].set_title(k)
        axs[i].set_xticklabels(tokens, rotation=90, ha='center')


        axs[i].set_title(k)

    fig.suptitle(exp_name, fontsize=16)
    plt.tight_layout(pad=1.0)
    if DEMO:
        plt.show()
    else:
        plt.savefig(path, format="png")
    plt.close()


/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/3422388298.py:14: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[i].set_xticklabels(tokens, rotation=90, ha='center')
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/3422388298.py:14: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[i].set_xticklabels(tokens, rotation=90, ha='center')
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/3422388298.py:14: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[i].set_xticklabels(tokens, rotation=90, ha='center')
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/3422388298.py:14: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or u

### Most popular

In [12]:
save_mp_path = os.path.join(EXPERIMENTS_PATH, "most_popular")
if os.path.exists(save_mp_path):
    print("Path already exists")
else:
    os.mkdir(save_mp_path)
    print("Path created")

Path already exists


In [13]:
for exp_name, report in reports.items():
    rep_mp = get_most_popular(report, n=NUMBER_OF_IMPORTANT_WC)
    _name = f"{exp_name}_mp.png"
    path = os.path.join(save_mp_path, _name)
    fig, axs = plt.subplots(1, len(rep_mp), figsize=(4*len(rep_mp), 4))
    for i, k in enumerate(rep_mp.keys()):
        wordcloud = WordCloud(width=800, height=800, background_color="white")
        wordcloud.generate_from_frequencies(dict(rep_mp[k]))

        axs[i].imshow(wordcloud, interpolation="bilinear")
        axs[i].axis("off")

        axs[i].set_title(k)

    fig.suptitle(exp_name, fontsize=16)
    plt.tight_layout(pad=1.0)
    if DEMO:
        plt.show()
    else:
        plt.savefig(path, format="png")
    plt.close()


## for particular set of tokens

In [14]:
EXP_and_NAME = {
    "effect on Label 0": "l0",
    "effect on Label 1": "l1",
    "effect on Label 2": "l2",
}

In [15]:
save_mp_path = os.path.join(EXPERIMENTS_PATH, "token effect")
if os.path.exists(save_mp_path):
    print("Path already exists")
else:
    os.mkdir(save_mp_path)
    print("Path created")

Path already exists


In [16]:
token_group_set = [
    ("punctuation", list(string.punctuation)),
    ("neutral tokens", ["a", "an", "what", "the", "you", "one", "it", "this", "movie", "i", "with", "if"]),
]


In [17]:
for token_group, token_set in token_group_set:
    _save_mp_path = os.path.join(save_mp_path, token_group)
    if not os.path.exists(_save_mp_path):
        os.mkdir(_save_mp_path)
        print("Path created")

    for exp_name, report in reports.items():
        rep_mp = get_importance(report, d_tokens=token_set)
        _name = f"{exp_name}_{token_group}.png"
        path = os.path.join(_save_mp_path, _name)
        rows = (len(EXP_and_NAME) + 2)//3
        fig, axs = plt.subplots(rows, 3, figsize=(20, 4*len(EXP_and_NAME)))

        for idx, exp in enumerate(EXP_and_NAME.keys()):
            i,j = idx//3, idx%3
            label_list = list(str(EXP_and_NAME[exp]).split('+'))
            temp_dict = dict()
            for ts in token_set:
                for l in label_list:
                    if rep_mp[ts][l]:
                        temp_dict[ts] = list(temp_dict.get(ts, [])) + list(rep_mp[ts][l])
            if rows > 1:
                axs[i][j].boxplot(temp_dict.values(), labels=temp_dict.keys())
                axs[i][j].set_title(exp)
            elif rows == 1:
                axs[j].boxplot(temp_dict.values(), labels=temp_dict.keys())
                axs[j].set_title(exp)
            else:
                print(f"Unknown case: {rows = }")

        fig.suptitle(exp_name, fontsize=16)
        plt.tight_layout(pad=1.0)
        if DEMO:
            plt.show()
        else:
            plt.savefig(path, format="png")
        plt.close()

In [18]:
_save_path = os.path.join(save_mp_path, "most important aggregated")
if not os.path.exists(_save_path):
    os.mkdir(_save_path)
    print("Path created")

for exp_name, report in reports.items():
    rep_mp = get_importance_all(report, metric_function=np.mean)
    _name = f"{exp_name}_aggr_bar.png"
    path = os.path.join(_save_path, _name)
    fig, axs = plt.subplots(1, len(rep_mp), figsize=(6*len(rep_mp), 6))

    for i, k in enumerate(rep_mp.keys()):
        tokens = [item[0] for item in rep_mp[k]]
        freqs = [item[1] for item in rep_mp[k]]

        axs[i].bar(tokens, freqs)
        axs[i].set_title(k)
        axs[i].set_xticklabels(tokens, rotation=90, ha='center')

        axs[i].set_title(k)

    fig.suptitle(exp_name, fontsize=16)
    plt.tight_layout(pad=1.0)
    if DEMO:
        plt.show()
    else:
        plt.savefig(path, format="png")
    plt.close()


/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/1041431682.py:18: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[i].set_xticklabels(tokens, rotation=90, ha='center')
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/1041431682.py:18: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[i].set_xticklabels(tokens, rotation=90, ha='center')
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/1041431682.py:18: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  axs[i].set_xticklabels(tokens, rotation=90, ha='center')
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/1041431682.py:18: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or u

## generate [token-mean-variance] dataset

In [19]:
from IPython.display import display, HTML
from pandas.plotting import table

_trends_path = os.path.join(save_mp_path, "trends")
_save_path = _trends_path
if not os.path.exists(_trends_path):
    os.mkdir(_trends_path)
    print("Path created")

for exp_name, report in reports.items():
    rep_mp = get_importance_all(
        report,
        metric_function=np.mean,
        split=False
    )
    rep_vp = get_importance_all(
        report,
        metric_function=np.var,
        split=False
    )
    rep_cp = get_importance_all(
        report,
        metric_function=len,
        split=False
    )
    _name = f"{exp_name}.csv"
    path = os.path.join(_trends_path, _name)
    img_path = os.path.join(_trends_path, exp_name + "-desc.png")

    tokens = [token for token, _ in rep_mp['l1']]
    data = [
        {"token": token, "mean0": 0, "mean1": 0, "mean2": 0, "variance0": 0, "variance1": 0, "variance2": 0, "count": 0} \
        for token in tokens
    ]

    for mm2, vv2, mm1, vv1, mm0, vv0, cc in zip(rep_mp["l2"], rep_vp["l2"], rep_mp["l1"], rep_vp["l1"], rep_mp["l0"], rep_vp["l0"], rep_cp["l0"]):
        idx_v = tokens.index(vv2[0])
        idx_m = tokens.index(mm2[0])
        data[idx_v]["variance2"] = vv2[1]
        data[idx_m]["mean2"] = mm2[1]

        idx_v = tokens.index(vv1[0])
        idx_m = tokens.index(mm1[0])
        data[idx_v]["variance1"] = vv1[1]
        data[idx_m]["mean1"] = mm1[1]

        idx_v = tokens.index(vv0[0])
        idx_m = tokens.index(mm0[0])
        data[idx_v]["variance0"] = vv0[1]
        data[idx_m]["mean0"] = mm0[1]

        idx_c = tokens.index(cc[0])
        data[idx_c]["count"] = cc[1]


    df = pd.DataFrame(data)

    fig, ax = plt.subplots(figsize=(4, 3))
    ax.axis('off')
    table(ax, df.describe(), loc='center')

    if DEMO:
        print(exp_name)
        display((df.describe()))
        # display(HTML(df.to_html(index=False)))
    else:
        df.to_csv(path, index=False)
        plt.savefig(img_path, dpi=300, bbox_inches='tight')
        plt.close()


## Tokens with zero variance

In [20]:
DEMO = False

In [21]:
trends_db_files = sorted(find_files(_trends_path, '.csv'))
threshold = 1e-2
sut = ["variance", "mean"][0]
N_time_token = 3  # tokens appeared less-equal of this

zero_vars: dict[set] = {}
all_token_count = 0

for dbf in trends_db_files:
    name = os.path.splitext(dbf)[0]

    df = pd.read_csv(os.path.join(_trends_path, dbf))
    all_token_count = max(all_token_count, len(df))
    df = df[(df[f'{sut}0'] <= threshold) & (df[f'{sut}1'] <= threshold) & (df[f'{sut}2'] <= threshold)]  # TODO: fix in to work for mean
    df.drop(columns=[f'{sut}0', f'{sut}1'], inplace=True)

    tokens = set(df["token"])
    zero_vars[name] = tokens

In [22]:
import json

sets_name = sorted(list(zero_vars.keys()))
df = [{"set": set_name, "new inserts": len(zero_vars[set_name]), "deleted": 0, "total": len(zero_vars[set_name]), "N times": 0} for set_name in sets_name]

ntc, curr = 0, sets_name[0]
_df = pd.read_csv(os.path.join(_trends_path, curr + ".csv"))
for tok in zero_vars[curr]:
    msk = _df["token"] == tok
    if sum(msk) == 0:
        continue
    count = int(_df[msk]["count"])
    if count <= N_time_token:
        ntc += 1
df[0]["N times"] = ntc

changes_dataset = {
    "epoch 0" : {
        "added": list(zero_vars[sets_name[0]]),
        "deleted": []
    },
}
always_exists = zero_vars[sets_name[0]]

for idx, (prev, curr) in enumerate(zip(sets_name[:-1], sets_name[1:]), start=1):
    intersects = zero_vars[prev].intersection(zero_vars[curr])
    # common_count = len(list(intersects))
    deleted = zero_vars[prev] - intersects
    added = zero_vars[curr] - intersects
    always_exists = always_exists.intersection(zero_vars[curr])

    epoch = f"epoch {idx}"
    changes_dataset[epoch] = {
        "added": list(added),
        "deleted": list(deleted),
    }

    df[idx]["new inserts"] = len(list(added))
    df[idx]["deleted"] = len(list(deleted))

    ntc = 0
    _df = pd.read_csv(os.path.join(_trends_path, curr + ".csv"))
    for tok in zero_vars[curr]:
        msk = _df["token"] == tok
        if sum(msk) == 0:
            continue
        count = int(_df[msk]["count"])
        if count <= N_time_token:
            ntc += 1
    df[idx]["N times"] = ntc

always_exists = list(always_exists)
changes_dataset["always exists"] = always_exists
df = pd.DataFrame(df)

log_path = os.path.join(_trends_path, f"log_{sut}.png")
change_data_path = os.path.join(_trends_path, f"token_changes_{sut}.json")

always_exists_count = len(always_exists)
ae_prompt = f"#Common tokens = {always_exists_count} of {all_token_count} ({(100*always_exists_count/all_token_count):.1f}%)"

if DEMO:
    display(HTML(df.to_html(index=False)))
    print(ae_prompt)
else:
    fig, ax = plt.subplots(figsize=(6, 2))
    ax.axis('off')
    plt.title(f"Token trends during tuning (threshold = {threshold}, {sut}, N={N_time_token})")
    plt.figtext(0.5, 0.01, ae_prompt, ha="center", fontsize=10, bbox={"facecolor":"blue", "alpha":0.2, "pad":2})
    table(ax, df, loc='center')
    plt.savefig(log_path, dpi=300, bbox_inches='tight')
    plt.close()

    with open(change_data_path, 'w') as f:
        json.dump(changes_dataset, f, indent=4)

/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/2813871278.py:12: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  count = int(_df[msk]["count"])
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/2813871278.py:47: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  count = int(_df[msk]["count"])
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/2813871278.py:47: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  count = int(_df[msk]["count"])
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/2813871278.py:47: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  count = int

### Most effective tokens set trend

In [24]:
trends_db_files = sorted(find_files(_trends_path, '.csv'))
proportion = 0.1
N_time_token = 3  # tokens appeared less-equal of this

highest_scores: dict[set] = {}
all_token_count = 0

for dbf in trends_db_files:
    name = os.path.splitext(dbf)[0]

    df = pd.read_csv(os.path.join(_trends_path, dbf))
    all_token_count = max(all_token_count, len(df))
    desired_df_split = int(len(df) * proportion)
    lowest_sorted_0 = (df['mean0'].sort_values(ascending=False).values)[desired_df_split]
    lowest_sorted_1 = (df['mean1'].sort_values(ascending=False).values)[desired_df_split]
    lowest_sorted_2 = (df['mean2'].sort_values(ascending=False).values)[desired_df_split]
    df = df[(df['mean0'] >= lowest_sorted_0) | (df['mean1'] >= lowest_sorted_1) | (df['mean2'] >= lowest_sorted_2)]


    tokens = set(df["token"])
    highest_scores[name] = tokens

In [25]:
import json

sets_name = sorted(list(highest_scores.keys()))
df = [{"set": set_name, "new inserts": len(highest_scores[set_name]), "deleted": 0, "total": len(highest_scores[set_name]), "N times": 0} for set_name in sets_name]

ntc, curr = 0, sets_name[0]
_df = pd.read_csv(os.path.join(_trends_path, curr + ".csv"))
for tok in highest_scores[curr]:
    msk = (_df["token"] == tok)
    if sum(msk) == 0:
        continue
    count = int(_df[msk]["count"])
    if count <= N_time_token:
        ntc += 1
df[0]["N times"] = ntc

changes_dataset = {
    "epoch 0" : {
        "added": list(highest_scores[sets_name[0]]),
        "deleted": []
    },
}
always_exists = highest_scores[sets_name[1]]

for idx, (prev, curr) in enumerate(zip(sets_name[:-1], sets_name[1:]), start=1):
    intersects = highest_scores[prev].intersection(highest_scores[curr])
    # common_count = len(list(intersects))
    deleted = highest_scores[prev] - intersects
    added = highest_scores[curr] - intersects
    always_exists = always_exists.intersection(highest_scores[curr])

    epoch = f"epoch {idx}"
    changes_dataset[epoch] = {
        "added": list(added),
        "deleted": list(deleted),
    }

    df[idx]["new inserts"] = len(list(added))
    df[idx]["deleted"] = len(list(deleted))

    ntc = 0
    _df = pd.read_csv(os.path.join(_trends_path, curr + ".csv"))
    for tok in highest_scores[curr]:
        msk = _df["token"] == tok
        if sum(msk) == 0:
            continue
        count = int(_df[msk]["count"])
        if count <= N_time_token:
            ntc += 1
    df[idx]["N times"] = ntc

always_exists = list(always_exists)
changes_dataset["always exists"] = always_exists
df = pd.DataFrame(df)

log_path = os.path.join(_trends_path, f"log_highest_scores_{proportion}.png")
commons_file_name = os.path.join(_trends_path, f"common_highest_scores_{proportion}.png")
change_data_path = os.path.join(_trends_path, f"token_changes_highest_scores_{proportion}.json")

always_exists_count = len(always_exists)
ae_prompt = f"#Common tokens = {always_exists_count} of {all_token_count} ({(100*always_exists_count/all_token_count):.1f}%)"

from collections import Counter
word_could_dict=Counter(always_exists)
wordcloud = WordCloud(
    width=1000,
    height=500,
    background_color="White",
    # colormap='Blues',
    relative_scaling=1,
).generate_from_frequencies(word_could_dict)

if DEMO:
    display(HTML(df.to_html(index=False)))
    print(ae_prompt)
    plt.figure(figsize=(15,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title("Common Highest score words")
    plt.show()
    plt.close()
else:
    fig, ax = plt.subplots(figsize=(6, 2))
    ax.axis('off')
    plt.title(f"Token with Highest scores during tuning (proportion = {proportion}, N={N_time_token})")
    plt.figtext(0.5, 0.01, ae_prompt, ha="center", fontsize=10, bbox={"facecolor":"blue", "alpha":0.2, "pad":2})
    table(ax, df, loc='center')
    plt.savefig(log_path, dpi=300, bbox_inches='tight')
    plt.close()

    with open(change_data_path, 'w') as f:
        json.dump(changes_dataset, f, indent=4)

    plt.figure(figsize=(15,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title("Common Highest score words")
    plt.savefig(commons_file_name, bbox_inches='tight')
    plt.close()


/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/522478166.py:12: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  count = int(_df[msk]["count"])
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/522478166.py:47: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  count = int(_df[msk]["count"])
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/522478166.py:47: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  count = int(_df[msk]["count"])
/var/folders/qr/bqqcrz2x32g6f0wk_w9qf8580000gn/T/ipykernel_12407/522478166.py:47: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  count = int(_df